In [68]:
import pandas as pd
import numpy as np

from tsl.ops.similarities import geographical_distance


In [75]:
red_sites_df = pd.read_csv('..\data\\aqs_sites_reduced.csv')

dist = geographical_distance(red_sites_df.loc[:, ["Latitude","Longitude"]])
dist_npy = dist.to_numpy()
np.save('..\data\prova.npy', dist_npy)
dist_mat = pd.DataFrame(np.load('..\data\prova.npy', allow_pickle=True))
dist_mat



,0,1,2,3,4,5,6,7,8,9,...,20747,20748,20749,20750,20751,20752,20753,20754,20755,20756
0,0.000000,2.926346,32.108792,9676.611868,240.051012,240.058687,253.665161,139.795039,118.588364,88.059774,...,2310.124300,2309.426022,2308.960437,2310.184694,2177.921452,2647.121383,2177.309158,1726.737896,2314.719617,2308.617148
1,2.926346,0.000000,34.366083,9673.829833,240.552921,240.569322,254.297445,136.882108,115.971529,90.871639,...,2312.930906,2312.230948,2311.767246,2312.990149,2180.720029,2649.981806,2176.074817,1726.100163,2317.017650,2310.911252
2,32.108792,34.366083,0.000000,9706.149445,216.166732,216.106015,228.722436,163.533484,134.228065,75.807803,...,2280.990872,2280.265217,2279.830372,2281.032464,2148.662052,2619.072555,2206.710232,1752.745285,2309.036108,2303.002818
3,9676.611868,9673.829833,9706.149445,0.000000,9787.992357,9788.639656,9804.473694,9548.424661,9591.827594,9749.272456,...,11985.798619,11985.230494,11984.618806,11985.949133,11854.149380,12309.295096,8216.813298,8812.137515,10955.000844,10947.577791
4,240.051012,240.552921,216.166732,9787.992357,0.000000,0.727555,17.690161,286.283913,235.123613,265.306746,...,2215.003506,2213.854914,2213.900648,2214.752293,2081.348177,2567.411732,2409.500989,1966.479135,2438.844779,2433.199816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20752,2647.121383,2649.981806,2619.072555,12309.295096,2567.411732,2566.714176,2552.419710,2780.660162,2740.912436,2570.080504,...,381.816360,384.730186,382.546475,383.346371,508.845584,0.000000,4304.895770,3698.841154,2093.339065,2096.464547
20753,2177.309158,2176.074817,2206.710232,8216.813298,2409.500989,2409.683372,2425.272929,2134.435564,2183.073677,2192.201167,...,4075.984563,4077.270942,4074.694938,4077.356950,3965.159165,4304.895770,0.000000,609.690802,2753.518068,2745.875222
20754,1726.737896,1726.100163,1752.745285,8812.137515,1966.479135,1966.520520,1980.395338,1714.689169,1758.387315,1723.230823,...,3477.731002,3479.164030,3476.441617,3479.196087,3369.634030,3698.841154,609.690802,0.000000,2184.858624,2177.093124
20755,2314.719617,2317.017650,2309.036108,10955.000844,2438.844779,2438.241527,2436.581803,2433.298197,2430.835178,2233.264958,...,2088.843629,2092.654750,2087.838347,2091.692523,2053.692581,2093.339065,2753.518068,2184.858624,0.000000,7.826790


In [35]:
dist_mat = pd.read_csv("../data/dist_matrix.csv")

dist_mat.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,20747,20748,20749,20750,20751,20752,20753,20754,20755,20756
0,0,0.000000,2.926346,32.108792,9676.611868,240.051012,240.058687,253.665161,139.795039,118.588364,...,2310.124300,2309.426022,2308.960437,2310.184694,2177.921452,2647.121383,2177.309158,1726.737896,2314.719617,2308.617148
1,1,2.926346,0.000000,34.366083,9673.829833,240.552921,240.569322,254.297445,136.882108,115.971529,...,2312.930906,2312.230948,2311.767246,2312.990149,2180.720029,2649.981806,2176.074817,1726.100163,2317.017650,2310.911252
2,2,32.108792,34.366083,0.000000,9706.149445,216.166732,216.106015,228.722436,163.533484,134.228065,...,2280.990872,2280.265217,2279.830372,2281.032464,2148.662052,2619.072555,2206.710232,1752.745285,2309.036108,2303.002818
3,3,9676.611868,9673.829833,9706.149445,0.000000,9787.992357,9788.639656,9804.473694,9548.424661,9591.827594,...,11985.798619,11985.230494,11984.618806,11985.949133,11854.149380,12309.295096,8216.813298,8812.137515,10955.000844,10947.577791
4,4,240.051012,240.552921,216.166732,9787.992357,0.000000,0.727555,17.690161,286.283913,235.123613,...,2215.003506,2213.854914,2213.900648,2214.752293,2081.348177,2567.411732,2409.500989,1966.479135,2438.844779,2433.199816


In [57]:
n = list((dist_mat.loc[:, ['0']].sort_values(by=["0"]))[:10].index)

KeyError: "None of [Index(['0'], dtype='object')] are in the [columns]"

In [13]:
nodes = n
nodes

[0,
 1,
 320,
 322,
 325,
 321,
 326,
 324,
 328,
 79,
 323,
 78,
 2,
 76,
 15,
 14,
 62,
 77,
 327,
 365]

In [37]:
a = dist_mat.loc[nodes, [str(n) for n in nodes]]
a.sort_index(axis=0, inplace=True)

a

,0,1,320,322,325,321,326,324,328,79,323,78,2,76,15,14,62,77,327,365
0,0.000000,2.926346,16.768443,17.247087,17.293145,17.362706,17.552134,17.657053,19.853242,23.156428,24.242615,25.197501,32.108792,32.273550,47.037821,47.303324,51.246266,53.213356,60.773629,74.015463
1,2.926346,0.000000,13.880186,14.360521,14.378360,14.437773,14.643044,14.863696,17.002772,21.316269,21.379952,23.278853,34.366083,29.838262,48.894235,49.196423,53.943362,50.935009,58.507636,72.962428
2,32.108792,34.366083,44.838745,47.562541,47.144305,46.376756,45.935382,48.661459,50.392573,55.164690,50.682671,57.245629,0.000000,64.186064,58.389607,58.175598,22.847936,85.289228,73.712854,103.710313
14,47.303324,49.196423,60.714262,57.961074,58.815344,60.066012,60.932549,56.612399,59.001485,49.233105,67.731297,50.610861,58.175598,60.322642,0.835036,0.000000,59.723664,71.873893,107.428765,65.833954
15,47.037821,48.894235,60.303656,57.499525,58.363862,59.631471,60.508674,56.129333,58.508827,48.628914,67.289259,49.987563,58.389607,59.674970,0.000000,0.835036,60.227019,71.107309,107.063512,65.009121
62,51.246266,53.943362,66.082742,68.061923,67.845167,67.375135,67.123166,68.788073,70.817962,72.928191,72.504207,75.087521,22.847936,83.289030,60.227019,59.723664,0.000000,103.634532,96.229531,116.297714
76,32.273550,29.838262,22.387312,17.854332,18.737501,20.299637,21.343806,16.114960,15.247604,11.861873,22.206398,10.009911,64.186064,0.000000,59.674970,60.322642,83.289030,21.336561,58.572172,51.354924
77,53.213356,50.935009,43.353327,39.084364,39.889377,41.317793,42.260660,37.432314,36.377420,30.709330,41.596813,28.548576,85.289228,21.336561,71.107309,71.873893,103.634532,0.000000,70.619297,38.555082
78,25.197501,23.278853,20.735405,15.685202,16.868314,18.759959,20.006775,13.446669,14.261422,2.162333,23.871688,0.000000,57.245629,10.009911,49.987563,50.610861,75.087521,28.548576,63.663049,51.379757
79,23.156428,21.316269,19.738527,14.754206,15.952924,17.846674,19.097822,12.520706,13.684071,0.000000,23.502092,2.162333,55.164690,11.861873,48.628914,49.233105,72.928191,30.709330,63.714594,52.792043


In [38]:
sites = pd.read_csv('../data/aqs_sites_reduced.csv')
sites.head()

,State Code,County Code,Site Number,Latitude,Longitude
0,01,1,1,32.437458,-86.472891
1,01,1,2,32.428470,-86.443585
2,01,1,3,32.332659,-86.791521
3,01,3,1,0.000000,0.000000
4,01,3,2,30.552367,-87.706911


In [39]:
sites.loc[nodes]

,State Code,County Code,Site Number,Latitude,Longitude
0,01,1,1,32.437458,-86.472891
1,01,1,2,32.428470,-86.443585
320,01,101,1,32.365695,-86.315802
322,01,101,3,32.406833,-86.292759
325,01,101,8,32.396249,-86.295247
321,01,101,2,32.379897,-86.300963
326,01,101,9,32.369027,-86.304413
324,01,101,7,32.425970,-86.285247
328,01,101,1002,32.412811,-86.263394
79,01,51,4,32.535680,-86.255193


In [40]:
from airquality import AirQuality, AirQualityGraph

In [41]:
DATA_DIR_PATH = 'data'
data_path = DATA_DIR_PATH
dist_mat_path = f'..\{data_path}\\dist_matrix.csv'

dist_mat_path

'..\\data\\dist_matrix.csv'

In [42]:
import os

os.path.exists(dist_mat_path)

True

In [34]:
aq = AirQuality()

Generating distance matrix... 

AttributeError: 'NoneType' object has no attribute 'loc'

In [47]:
dist_npy = dist_mat.to_numpy()

In [50]:
np.save('..\data\dist_mat.npy', dist_npy)

In [51]:
a = np.load('..\data\dist_mat.npy', allow_pickle=True)
a

array([[0.00000000e+00, 0.00000000e+00, 2.92634574e+00, ...,
        1.72673790e+03, 2.31471962e+03, 2.30861715e+03],
       [1.00000000e+00, 2.92634574e+00, 0.00000000e+00, ...,
        1.72610016e+03, 2.31701765e+03, 2.31091125e+03],
       [2.00000000e+00, 3.21087922e+01, 3.43660828e+01, ...,
        1.75274528e+03, 2.30903611e+03, 2.30300282e+03],
       ...,
       [2.07540000e+04, 1.72673790e+03, 1.72610016e+03, ...,
        0.00000000e+00, 2.18485862e+03, 2.17709312e+03],
       [2.07550000e+04, 2.31471962e+03, 2.31701765e+03, ...,
        2.18485862e+03, 0.00000000e+00, 7.82678996e+00],
       [2.07560000e+04, 2.30861715e+03, 2.31091125e+03, ...,
        2.17709312e+03, 7.82678996e+00, 0.00000000e+00]])

In [52]:
dist_mat = pd.DataFrame(a)
dist_mat.head()

,0,1,2,3,4,5,6,7,8,9,...,20748,20749,20750,20751,20752,20753,20754,20755,20756,20757
0,0.0,0.000000,2.926346,32.108792,9676.611868,240.051012,240.058687,253.665161,139.795039,118.588364,...,2310.124300,2309.426022,2308.960437,2310.184694,2177.921452,2647.121383,2177.309158,1726.737896,2314.719617,2308.617148
1,1.0,2.926346,0.000000,34.366083,9673.829833,240.552921,240.569322,254.297445,136.882108,115.971529,...,2312.930906,2312.230948,2311.767246,2312.990149,2180.720029,2649.981806,2176.074817,1726.100163,2317.017650,2310.911252
2,2.0,32.108792,34.366083,0.000000,9706.149445,216.166732,216.106015,228.722436,163.533484,134.228065,...,2280.990872,2280.265217,2279.830372,2281.032464,2148.662052,2619.072555,2206.710232,1752.745285,2309.036108,2303.002818
3,3.0,9676.611868,9673.829833,9706.149445,0.000000,9787.992357,9788.639656,9804.473694,9548.424661,9591.827594,...,11985.798619,11985.230494,11984.618806,11985.949133,11854.149380,12309.295096,8216.813298,8812.137515,10955.000844,10947.577791
4,4.0,240.051012,240.552921,216.166732,9787.992357,0.000000,0.727555,17.690161,286.283913,235.123613,...,2215.003506,2213.854914,2213.900648,2214.752293,2081.348177,2567.411732,2409.500989,1966.479135,2438.844779,2433.199816


In [76]:
type(dist_mat)

pandas.core.frame.DataFrame

In [77]:
dist_mat.shape

(20757, 20757)

In [97]:
a = [4,5,3]
a.sort()
a

[3, 4, 5]

In [95]:
(dist_mat.loc[[3,2,1],[3,2,1]]).sort_index(axis=0)

,3,2,1
1,9673.829833,34.366083,0.000000
2,9706.149445,0.000000,34.366083
3,0.000000,9706.149445,9673.829833


In [110]:
closest_dist = (dist_mat.loc[:, [75]].sort_values(by=[75]))[1:10+1]


In [109]:
closest_dist

,75
74,0.051871
71,1.095910
67,24.854606
69,24.856519
70,27.032018
68,27.059239
64,27.565034
84,27.791816
72,27.817779
